In [11]:
import numpy as np
import tinygrad 
from extra.datasets import fetch_cifar
import time
import cv2
start =time.monotonic()
X_train, Y_train,= fetch_cifar(train=True)

In [12]:
from tinygrad.nn import Linear
from tinygrad.nn import optim
from tinygrad.tensor import Tensor

class TinyCIFAR:
    def __init__ (self):
        self.l1 = Linear(3072,1024, bias=True)
        self.l2 = Linear(1024,1024,bias=True)
        self.l3 = Linear(1024,10, bias=True)

    def __call__ (self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        x = x.leakyrelu()
        x = self.l3(x)
        return x.log_softmax()
    
net = TinyCIFAR()

In [13]:
print(X_train.shape)
X_trains = X_train.reshape(50000, -1)
print(X_trains.shape)
Tensor.training= True

from tinygrad.nn.optim import SGD, Adam
opt = Adam([net.l1.weight, net.l2.weight, net.l3.weight], lr=3e-4)

def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

(50000, 3, 32, 32)
(50000, 3072)


In [14]:



BS=64

#Train the modelieren

for step in range(1001):
    samp=np.random.randint(0, X_trains.shape[0], BS)
    batch = Tensor(X_trains[0], requires_grad=True)
    labels = Y_train[samp]
    out = net(batch)
    loss = cross_entropy(out, labels)
    #print(loss.numpy())
    opt.zero_grad() #NOTE:I'm unsure why it's giving me t.grad is not None (it was array shaping)
    loss.backward()
    opt.step()
    #accuracy dealings
    pred = np.argmax(out.numpy(), axis=-1)
    acc = (pred == labels).mean()
    # if acc >= 1:
    #     break
    if step % 100 == 0:
        print(f'step{step} | loss: {loss.numpy()} | Accuracy: {acc}')



step0 | loss: 118.45318603515625 | Accuracy: 0.109375
step100 | loss: 9.610190391540527 | Accuracy: 0.109375
step200 | loss: 3.1440982818603516 | Accuracy: 0.109375
step300 | loss: 2.9679391384124756 | Accuracy: 0.09375
step400 | loss: 2.920276165008545 | Accuracy: 0.046875
step500 | loss: 2.7494921684265137 | Accuracy: 0.109375
step600 | loss: 2.5133960247039795 | Accuracy: 0.109375
step700 | loss: 2.490748167037964 | Accuracy: 0.015625
step800 | loss: 2.4445998668670654 | Accuracy: 0.078125
step900 | loss: 2.4531660079956055 | Accuracy: 0.09375
step1000 | loss: 2.3521833419799805 | Accuracy: 0.125


In [26]:
#Validate model
X_test, Y_test= fetch_cifar(train=False)
print(X_test.shape, Y_test.shape)
av_acc = 0 #loopy fucks up without this
X_tests = X_test.reshape(10000, -1)
print(X_tests.shape)
testamount = 100
for step in range(testamount):
    samp = np.random.randint(0, X_tests.shape[0], size=BS)
    batch = Tensor(X_tests[samp], requires_grad=True)
    labels = Y_test[samp]
    out = net (batch)

    pred = np.argmax(out.numpy(), axis=-1)
    av_acc += (pred == labels).mean()
    # print(out.numpy())
    # #print(np.argmax(out.numpy()))
    # print(Y_test[samp])
    # #print(batch.numpy())

print(f"Test Accuracy: {av_acc / testamount}")

(10000, 3, 32, 32) (10000,)
(10000, 3072)
Test Accuracy: 0.095
